<a href="https://colab.research.google.com/github/fabriziobasso/Colab_backup/blob/main/File_00_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **<h1 align="center"><font color='#001ddd'> GLUCOSE PREDICTION DATASET**</font></h1>

## **Dataset Description**
The dataset is from a study that collected data from young adults in the UK with type 1 diabetes, who used a continuous glucose monitor (CGM), an insulin pump and a smartwatch. These devices collected blood glucose readings, insulin dosage, carbohydrate intake, and activity data. The data collected was aggregated to five-minute intervals and formatted into samples. Each sample represents a point in time and includes the aggregated five-minute intervals from the previous six hours. The aim is to predict the blood glucose reading an hour into the future, for each of these samples.

The training set takes samples from the first three months of study data from nine of the participants and includes the future blood glucose value. These training samples appear in chronological order and overlap. The testing set takes samples from the remainder of the study period from fifteen of the participants (so unseen participants appear in the testing set). These testing samples do not overlap and are in a random order to avoid data leakage.

**Complexities to be aware of:**

This is medical data so there are missing values and noise in the data
the participants did not all use the same device models (CGM, insulin pump and smartwatch) so there may be differences in the collection method of the data
some participants in the test set do not appear in the training set

In [ ]:
%%capture
# Connect to Colab:#
from google.colab import drive
import os
drive.mount('/content/drive')

!pip install category-encoders
!pip install optuna
!pip install optuna-integration
#!pip install scikit-learn==1.4
!pip install catboost
!pip install deeptables

!pip install BorutaShap
!pip install scikit-lego
!!pip install --no-index -U --find-links=/kaggle/input/deeptables-v0-2-5/deeptables-0.2.5 deeptables==0.2.5

In [ ]:
folder_script = models_folders = "/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose"
os.chdir(folder_script)

In [ ]:
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.wrapper import PolynomialWrapper
from category_encoders.count import CountEncoder

# Setup notebook
from pathlib import Path
import ipywidgets as widgets
import pandas as pd
import numpy as np
from pickle import load, dump
import json
import joblib
#import calplot as cal
import missingno as msno
import category_encoders as ce

# Graphic Libraries:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

# Palette Setup
colors = ['#FB5B68','#FFEB48','#2676A1','#FFBDB0',]
colormap_0 = mpl.colors.LinearSegmentedColormap.from_list("",colors)
palette_1 = sns.color_palette("coolwarm", as_cmap=True)
palette_2 = sns.color_palette("YlOrBr", as_cmap=True)
palette_3 = sns.light_palette("red", as_cmap=True)
palette_4 = sns.color_palette("viridis", as_cmap=True)
palette_5 = sns.color_palette("rocket", as_cmap=True)
palette_6 = sns.color_palette("GnBu", as_cmap=True)
palette_7 = sns.color_palette("tab20c", as_cmap=False)
palette_8 = sns.color_palette("Set2", as_cmap=False)

palette_custom = ['#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6','#ffffcc','#e5d8bd','#fddaec','#f2f2f2']
palette_9 = sns.color_palette(palette_custom, as_cmap=False)


# Bloomberg
#from xbbg import blp
from catboost import CatBoostRegressor, Pool
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from xgboost.callback import EarlyStopping

import lightgbm as lgb
from lightgbm import (LGBMRegressor,
                      LGBMClassifier,
                      early_stopping,
                      record_evaluation,
                      log_evaluation)

# Time Management
from tqdm import tqdm
from datetime import date
from datetime import datetime
from pandas.tseries.offsets import BMonthEnd, QuarterEnd
import datetime
from pandas.tseries.offsets import BDay # BDay is business day, not birthday...
import datetime as dt
import click
import glob
import os
import gc
import re
import string

from ipywidgets import AppLayout
from ipywidgets import Dropdown, Layout, HTML, AppLayout, VBox, Label, HBox, BoundedFloatText, interact, Output

#from my_func import *

import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_contour

os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf
import keras
from keras import ops
from keras import layers

from keras.layers import Input, LSTM, Dense, Lambda, RepeatVector, Reshape
from keras.models import Model
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError, MeanAbsoluteError

from keras.utils import FeatureSpace, plot_model

#from my_func import *

# preprocessing modules
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, RepeatedKFold, cross_validate, GroupKFold, GridSearchCV, RepeatedStratifiedKFold, cross_val_predict

from sklearn.preprocessing import (LabelEncoder,
                                   StandardScaler,
                                   MinMaxScaler,
                                   OrdinalEncoder,
                                   RobustScaler,
                                   PowerTransformer,
                                   OneHotEncoder,
                                   LabelEncoder,
                                   QuantileTransformer,
                                   PolynomialFeatures)

# metrics
import sklearn
from sklearn.metrics import (mean_squared_error,
                             root_mean_squared_error,
                             r2_score,
                             mean_absolute_error,
                             mean_absolute_percentage_error,
                             classification_report,
                             confusion_matrix,
                             ConfusionMatrixDisplay,
                             multilabel_confusion_matrix,
                             accuracy_score,
                             roc_auc_score,
                             auc,
                             roc_curve,
                             log_loss,
                             make_scorer)

# modeling algos
from sklearn.linear_model import (LogisticRegression,
                                  Lasso,
                                  ridge_regression,
                                  LinearRegression,
                                  Ridge,
                                  RidgeCV,
                                  ElasticNet,
                                  BayesianRidge,
                                  HuberRegressor,
                                  TweedieRegressor,
                                  QuantileRegressor,
                                  ARDRegression,
                                  TheilSenRegressor,
                                  PoissonRegressor,
                                  GammaRegressor)

from sklearn.ensemble import (AdaBoostRegressor,
                              AdaBoostClassifier,
                              RandomForestRegressor,
                              RandomForestClassifier,
                              VotingRegressor,
                              GradientBoostingRegressor,
                              GradientBoostingClassifier,
                              StackingRegressor,
                              HistGradientBoostingClassifier,
                              HistGradientBoostingRegressor,
                              ExtraTreesClassifier)

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import FunctionTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline

import seaborn as sns
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

from sklearn.multioutput import RegressorChain
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

import itertools
import warnings
import logging
from openpyxl import load_workbook

import statsmodels.api as sm
from pylab import rcParams
import scipy.stats as ss

warnings.filterwarnings('ignore')
#plt.style.use('fivethirtyeight')

# Setting rc parameters in seaborn for plots and graphs-
# Reference - https://matplotlib.org/stable/tutorials/introductory/customizing.html:-
# To alter this, refer to matplotlib.rcParams.keys()

sns.set({"axes.facecolor"       : "#ffffff",
         "figure.facecolor"     : "#ffffff",
         "axes.edgecolor"       : "#000000",
         "grid.color"           : "#ffffff",
         "font.family"          : ['Cambria'],
         "axes.labelcolor"      : "#000000",
         "xtick.color"          : "#000000",
         "ytick.color"          : "#000000",
         "grid.linewidth"       : 0.5,
         'grid.alpha'           :0.5,
         "grid.linestyle"       : "--",
         "axes.titlecolor"      : 'black',
         'axes.titlesize'       : 12,
         'axes.labelweight'     : "bold",
         'legend.fontsize'      : 7.0,
         'legend.title_fontsize': 7.0,
         'font.size'            : 7.5,
         'xtick.labelsize'      : 7.5,
         'ytick.labelsize'      : 7.5,
        });

sns.set_style("whitegrid",{"grid.linestyle":"--", 'grid.linewidth':0.2, 'grid.alpha':0.5})
# Set Style
mpl.rcParams['figure.dpi'] = 120;

# Making sklearn pipeline outputs as dataframe:-
pd.set_option('display.max_columns', 100);
pd.set_option('display.max_rows', 50);

sns.despine(left=True, bottom=True, top=False, right=False)

mpl.rcParams['axes.spines.left'] = True
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.bottom'] = True

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


<Figure size 960x660 with 0 Axes>

In [ ]:
tf.__version__

'2.17.0'

In [ ]:
!apt install graphviz
import os
os.environ["PATH"] += os.pathsep + '/usr/bin/dot'

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


# 1.0 Upload Data

## 1.1 Functions:

In [ ]:
def encode_target(y_train, y_test, encoder_type='label', enc_strategy=False):
    """
    Encodes the target columns in the training and testing data
    using the specified encoder type.

    Parameters:
    y_train (pd.Series or pd.DataFrame): Training target data.
    y_test (pd.Series or pd.DataFrame): Testing target data.

    Returns:
    y_train_encoded (pd.Series): Encoded training target data.
    y_test_encoded (pd.Series): Encoded testing target data.
    """

    if encoder_type == 'label':
        encoder = LabelEncoder()
        y_train_encoded = encoder.fit_transform(y_train)
        y_test_encoded = encoder.transform(y_test)

        y_train_encoded = pd.Series(y_train_encoded, index=y_train.index, name="Target")
        y_test_encoded = pd.Series(y_test_encoded, index=y_test.index, name="Target")


    elif encoder_type == 'onehot':
        y_train_ = y_train.values.reshape(-1, 1)
        y_test_ = y_test.values.reshape(-1, 1)

        encoder = OneHotEncoder(sparse_output=False)
        y_train_encoded = encoder.fit_transform(y_train_)
        y_test_encoded = encoder.transform(y_test_)

        y_train_encoded = pd.DataFrame(y_train_encoded, index=y_train.index)
        y_test_encoded = pd.DataFrame(y_test_encoded, index=y_test.index)

    else:
        raise ValueError("Invalid encoder_type. Currently supported: 'label'.")

    if enc_strategy:
        return y_train_encoded, y_test_encoded, encoder

    else:
        return y_train_encoded, y_test_encoded

def encode_data(X_train, X_test, encoder_type='label', columns=None, map=None):
    """
    Encodes the training and testing data using the specified encoder type.

    Parameters:
    X_train (pd.DataFrame): Training data.
    X_test (pd.DataFrame): Testing data.
    encoder_type (str): Type of encoder ('label' or 'onehot'). Default is 'label'.
    columns (list): List of columns to encode. If None, all object type columns are encoded.

    Returns:
    X_train_encoded (pd.DataFrame): Encoded training data.
    X_test_encoded (pd.DataFrame): Encoded testing data.
    """

    if columns is None:
        # Default to all object type columns if no columns are specified
        columns = X_train.select_dtypes(include=['object']).columns.tolist()

    X_train_encoded = X_train.copy()
    X_test_encoded = X_test.copy()

    if encoder_type == 'label':
        for col in columns:
            le = LabelEncoder()
            X_train_encoded[col] = le.fit_transform(X_train[col])
            X_test_encoded[col] = le.transform(X_test[col])

    elif encoder_type == 'onehot':
        for col in columns:
            ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first')
            # Fit the encoder on the training data and transform both training and test data
            encoded_train = ohe.fit_transform(X_train[[col]])
            encoded_test = ohe.transform(X_test[[col]])

            # Create a DataFrame with the encoded data
            encoded_train_df = pd.DataFrame(encoded_train, columns=ohe.get_feature_names_out([col]))
            encoded_test_df = pd.DataFrame(encoded_test, columns=ohe.get_feature_names_out([col]))

            # Concatenate the new columns to the original dataframes and drop the original columns
            X_train_encoded = pd.concat([X_train_encoded.drop(col, axis=1), encoded_train_df], axis=1)
            X_test_encoded = pd.concat([X_test_encoded.drop(col, axis=1), encoded_test_df], axis=1)

    elif encoder_type == 'count_encoder':

          for col in columns:

                target_encoder = CountEncoder(cols=columns)
                X_train_encoded = target_encoder.fit_transform(X_train_encoded)
                X_test_encoded = target_encoder.transform(X_test_encoded)

    else:
        raise ValueError("Invalid encoder_type. Currently supported: 'label', 'onehot', 'target_encoder'.")

    return X_train_encoded, X_test_encoded

def plot_training_session(history):
  # Plot training and validation loss scores
  # against the number of epochs.
  plt.figure(figsize=(8, 6))
  plt.plot(history.history['loss'], label='Train')
  plt.plot(history.history['val_loss'], label='Validation')
  plt.grid(linestyle='--')
  plt.ylabel('val_loss')
  plt.xlabel('Epoch')
  plt.title('Train-Validation Scores', pad=13)
  plt.legend(loc='upper right');
  plt.show()

## **1.2 Importing the Dataset**

### **1.2.1 Files**
* activities.txt - a list of activity names that appear in the activity-X:XX columns
* sample_submission.csv - a sample submission file in the correct format
* test.csv - the test set
* train.csv - the training set

## **Columns**
* train.csv:
    * **id - row id** consisting of participant number and a count for that participant
    * **p_num** - participant number
    * **time** - time of day in the format HH:MM:SS
    * **bg-X:XX** - blood glucose reading in mmol/L, X:XX(H:SS) time in the past (e.g. bg-2:35, would be the blood glucose reading from 2 hours and 35 minutes before the time value for that row), recorded by the continuous glucose monitor
    * **insulin-X:XX** - total insulin dose received in units in the last 5 minutes, X:XX(H:SS) time in the past (e.g. insulin-2:35, would be the total insulin dose received between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), recorded by the insulin pump
    * **carbs-X:XX** - total carbohydrate value consumed in grammes in the last 5 minutes, X:XX(H:SS) time in the past (e.g. carbs-2:35, would be the total carbohydrate value consumed between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), recorded by the participant
    * **hr-X:XX** - mean heart rate in beats per minute in the last 5 minutes, X:XX(H:SS) time in the past (e.g. hr-2:35, would be the mean heart rate between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), recorded by the smartwatch
    * **steps-X:XX** - total steps walked in the last 5 minutes, X:XX(H:SS) time in the past (e.g. * steps-2:35, would be the total steps walked between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), recorded by the smartwatch
    * **cals-X:XX** - total calories burnt in the last 5 minutes, X:XX(H:SS) time in the past (e.g. cals-2:35, would be the total calories burned between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), calculated by the smartwatch
    * **activity-X:XX** - self-declared activity performed in the last 5 minutes, X:XX(H:SS) time in the past (e.g. activity-2:35, would show a string name of the activity performed between 2 hours and 40 minutes and 2 hours and 35 minutes before the time value for that row), set on the smartwatch
    * **bg+1:00** - blood glucose reading in mmol/L an hour in the future, this is the value you will be predicting (not provided in test.csv)

In [ ]:
ext_data=True

if ext_data==False:
  df_train=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/train_cluster.csv", index_col=0)
  df_test=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/test_cluster.csv", index_col=0)

  df_train_scaled=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/X_train_scaled.csv", index_col=0)
  df_test.shape

if ext_data==True:
  df_train = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/train_cluster_enc_final.csv", index_col=0)
  df_test = pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/test_cluster_enc_final.csv", index_col=0)

  df_train_scaled=pd.read_csv("/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/Glucose/X_train_scaled.csv", index_col=0)
  print(df_test.shape,df_train.shape)

(3644, 669) (177024, 670)


In [ ]:
df_train.p_num.value_counts()

,count
p_num,
p03,26028
p02,25872
p10,25454
p12,25299
p04,24686
p11,24555
p01,8459
p06,8383
p05,8288


In [ ]:
df_train.p_num = df_train.p_num.astype("str")
df_test.p_num = df_test.p_num.astype("str")

In [ ]:
df_train.head()
df_train.columns

Index(['p_num', 'hour', 'minute', 'obv_n', 'brake-5:55', 'brake-5:50',
       'brake-5:45', 'brake-5:40', 'brake-5:35', 'brake-5:30',
       ...
       'enc_07_v7', 'enc_01_v4', 'enc_05_v4', 'enc_03_v6', 'enc_05_v7',
       'enc_06_v6', 'enc_01_v6', 'enc_06_v7', 'enc_08_v6', 'enc_04_v6'],
      dtype='object', length=670)

In [ ]:
print(df_train.shape)
display(dict(df_train.groupby("p_num")["p_num"].count()))
display(dict(df_test.groupby("p_num")["p_num"].count()))
#display(df_train[df_train["p_num"]=="p01"].head())
#display(df_train[df_train["p_num"]=="p02"].head())

(177024, 670)


{'p01': 8459,
 'p02': 25872,
 'p03': 26028,
 'p04': 24686,
 'p05': 8288,
 'p06': 8383,
 'p10': 25454,
 'p11': 24555,
 'p12': 25299}

{'p01': 244,
 'p02': 227,
 'p04': 258,
 'p05': 276,
 'p06': 234,
 'p10': 179,
 'p11': 221,
 'p12': 288,
 'p15': 294,
 'p16': 248,
 'p18': 231,
 'p19': 246,
 'p21': 236,
 'p22': 201,
 'p24': 261}

###  **1.2.2 Data Preparation:**

Data are scaled in groups.

In [ ]:
df_train.columns[-20:],df_train.columns[:20]

(Index(['activity-0:05', 'activity-0:00', 'bg+1:00', 'cluster', 'cluster_pca',
        'PC_1', 'PC_2', 'PC_3', 'enc_04_v7', 'enc_01_v1', 'enc_07_v7',
        'enc_01_v4', 'enc_05_v4', 'enc_03_v6', 'enc_05_v7', 'enc_06_v6',
        'enc_01_v6', 'enc_06_v7', 'enc_08_v6', 'enc_04_v6'],
       dtype='object'),
 Index(['p_num', 'hour', 'minute', 'obv_n', 'brake-5:55', 'brake-5:50',
        'brake-5:45', 'brake-5:40', 'brake-5:35', 'brake-5:30', 'brake-5:25',
        'brake-5:20', 'brake-5:15', 'brake-5:10', 'brake-5:05', 'brake-5:00',
        'brake-4:55', 'brake-4:50', 'brake-4:45', 'brake-4:40'],
       dtype='object'))

In [ ]:
X_train = df_train.drop(columns=["p_num","obv_n"]).copy()
X_test = df_test.drop(columns=["p_num","obv_n"]).copy()

#X_train = df_train.drop(columns=["obv_n"]).copy()
#X_test = df_test.drop(columns=["obv_n"]).copy()

In [ ]:
hr_col_train = list(df_train.filter(regex='hr').columns)
print(len(hr_col_train))

X_train = X_train.drop(columns=hr_col_train)
X_test = X_test.drop(columns=hr_col_train)

72


In [ ]:
static_fields = ["hour","minute", 'cluster', 'cluster_pca',
       'PC_1', 'PC_2', 'PC_3', 'enc_04_v7', 'enc_01_v1', 'enc_07_v7',
       'enc_01_v4', 'enc_05_v4', 'enc_03_v6', 'enc_05_v7', 'enc_06_v6',
       'enc_01_v6', 'enc_06_v7', 'enc_08_v6', 'enc_04_v6'] #"id",
target = ["bg+1:00"]
ts_fields = list(X_train.drop(columns=static_fields+target))
len(ts_fields)

576

### Scale Features

##### Target:

In [ ]:
target_scaler = StandardScaler()
target_scaler.fit(df_train_scaled[target])

X_train[target] = target_scaler.transform(df_train[target])

In [ ]:
X_test[target] = np.nan

X_train = X_train[list(X_test.columns)]

In [ ]:
(X_train.columns==X_test.columns).all()

True

##### Features:

##### Define Features Datatypes:

In [ ]:
X_train[["cluster","cluster_pca"]] = X_train[["cluster","cluster_pca"]].astype("int32")

In [ ]:
int_cols = X_train.select_dtypes(include=['int']).columns.tolist()
float_cols = X_train.select_dtypes(include=['float']).columns.tolist()
obj_cols = X_train.select_dtypes(include=['object']).columns.tolist()

X_train[int_cols] = X_train[int_cols].astype("int32")
X_train[float_cols] = X_train[float_cols].astype("float32")
X_train[obj_cols] = X_train[obj_cols].astype("string")

X_test[int_cols] = X_test[int_cols].astype("int32")
X_test[float_cols] = X_test[float_cols].astype("float32")
X_test[obj_cols] = X_test[obj_cols].astype("string")

##### Time Features:

In [ ]:
X_train['hour_sin'] = np.sin(2 * np.pi * X_train['hour'] / 24)
X_train['hour_cos'] = np.cos(2 * np.pi * X_train['hour'] / 24)
X_train['minute_sin'] = np.sin(2 * np.pi * X_train['minute'] / 60)
X_train['minute_cos'] = np.cos(2 * np.pi * X_train['minute'] / 60)

X_train = X_train.drop(['hour', 'minute'], axis=1)

In [ ]:
X_test['hour_sin'] = np.sin(2 * np.pi * X_test['hour'] / 24)
X_test['hour_cos'] = np.cos(2 * np.pi * X_test['hour'] / 24)
X_test['minute_sin'] = np.sin(2 * np.pi * X_test['minute'] / 60)
X_test['minute_cos'] = np.cos(2 * np.pi * X_test['minute'] / 60)

X_test = X_test.drop(['hour', 'minute'], axis=1)

In [ ]:
ts_fields_group = list({q[:-5] for q in ts_fields})
ts_fields_group

['cals_av',
 'activity',
 'insulin_av',
 'bg',
 'brake',
 'carbs_av',
 'intake',
 'steps_av']

In [ ]:
X_train = X_train.copy()
X_test = X_test.copy()

In [ ]:
X_train.shape, X_test.shape

((177024, 598), (3644, 598))

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177024 entries, p01_0 to p12_25298
Columns: 598 entries, brake-5:55 to minute_cos
dtypes: float32(592), float64(4), int32(2)
memory usage: 407.9+ MB


## **DATASET SELECTION**

In [ ]:
X_train.shape, X_test.shape

((177024, 598), (3644, 598))

In [ ]:
Xt = X_train[['hour_sin','hour_cos','minute_sin','minute_cos', 'cluster_pca','PC_1', 'PC_2', 'PC_3',
              'enc_04_v7', 'enc_01_v1', 'enc_07_v7', 'enc_01_v4', 'enc_05_v4', 'enc_03_v6', 'enc_05_v7',
              'enc_06_v6', 'enc_01_v6', 'enc_06_v7', 'enc_08_v6', 'enc_04_v6',"bg-0:00", "insulin_av-0:00",
              "brake-0:00","intake-0:00",'steps_av-0:00']]

Xtest = X_test[['hour_sin','hour_cos','minute_sin','minute_cos', 'cluster_pca','PC_1', 'PC_2', 'PC_3',
                'enc_04_v7', 'enc_01_v1', 'enc_07_v7', 'enc_01_v4', 'enc_05_v4', 'enc_03_v6', 'enc_05_v7',
                'enc_06_v6', 'enc_01_v6', 'enc_06_v7', 'enc_08_v6', 'enc_04_v6',"bg-0:00", "insulin_av-0:00",
                "brake-0:00","intake-0:00",'steps_av-0:00']]

y = X_train[target].copy()

## **TREE MODELS**

#### **FUNCTIONS**

In [ ]:
def objective_xgb(trial, model_class, X, y, use_gpu=True, sc=target_scaler,n_splits=3,n_repeats=3):

    model_class = XGBRegressor

    categorical_features = []
    cat_bin = ['cluster_pca']
    tot_cat = categorical_features + cat_bin
    numeric_features = [col for col in X.columns if col not in tot_cat]

    params = {
    'n_estimators': 851, #trial.suggest_int('n_estimators', 100, 400, step=1),
    'learning_rate': 0.01, #trial.suggest_loguniform('learning_rate', 0.01, 0.03),
    'max_depth': trial.suggest_int('max_depth', 3, 15),
    'min_child_weight': trial.suggest_int('min_child_weight', 1, 25),
    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.50, 0.95, step=0.025),
    'subsample' :trial.suggest_float('subsample', .50, 0.95, step=0.025),
    'reg_lambda' : trial.suggest_float('reg_lambda', 1e-5, 1.0, log = True),
    'reg_alpha' :  trial.suggest_float('reg_alpha', 1e-5, 1.0, log = True),
    'gamma': trial.suggest_loguniform('gamma', 1e-5, 1.0),
    'random_state': 42,
    'tree_method': 'gpu_hist' if use_gpu else 'hist'}

    # Create the early stopping callback
    early_stop = EarlyStopping(rounds=61, metric_name="rmse")

    model = model_class(**params, objective='reg:squarederror', eval_metric='rmse', callbacks=[early_stop], enable_categorical="True")

    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', ce.TargetEncoder(), categorical_features),
            ('cat_bin', OneHotEncoder(drop="first",sparse_output=False,handle_unknown="ignore"), cat_bin),
            ('num', StandardScaler(), numeric_features)
        ])

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)], verbose=False)

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    rmse_scores = []

    for num, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        pipeline[:-1].fit(X_train, y_train)
        X_val_ = pipeline[:-1].transform(X_val) # Transform X_val using the same steps as X
        eval_set = [(X_val_, y_val)]
        fit_params = {'model__eval_set': eval_set, 'model__verbose': False}

        # Fit the pipeline with early stopping
        pipeline.fit(X_train, y_train, **fit_params)

        y_pred = pipeline.predict(X_val)

        y_val_ = sc.inverse_transform(y_val)
        y_pred_ = sc.inverse_transform(y_pred.reshape(-1, 1))

        print(f"Fold {num} RMSE: {np.sqrt(mean_squared_error(y_val_, y_pred_))}")
        rmse_scores.append(np.sqrt(mean_squared_error(y_val_, y_pred_)))
        gc.collect()

    return rmse_scores[0]

In [ ]:
def objective_xgb(trial, model_class, X, y, use_gpu=True, sc=target_scaler,n_splits=3,n_repeats=3):

    model_class = XGBRegressor

    categorical_features = []
    cat_bin = ['cluster_pca']
    tot_cat = categorical_features + cat_bin
    numeric_features = [col for col in X.columns if col not in tot_cat]

    params = {
    'n_estimators': 501, #trial.suggest_int('n_estimators', 100, 400, step=1),
    'learning_rate': 0.01, #trial.suggest_loguniform('learning_rate', 0.01, 0.03),
    'max_depth': trial.suggest_int('max_depth', 12, 20),
    'min_child_weight': trial.suggest_int('min_child_weight', 1, 12),
    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.55, 0.975, step=0.025),
    'subsample' :trial.suggest_float('subsample', .55, 0.975, step=0.025),
    'reg_lambda' : trial.suggest_float('reg_lambda', 1e-5, 1.0, log = True),
    'reg_alpha' :  trial.suggest_float('reg_alpha', 1e-5, 1.0, log = True),
    'gamma': trial.suggest_loguniform('gamma', 1e-5, 1.0),
    'random_state': 42,
    'tree_method': 'gpu_hist' if use_gpu else 'hist'}

    # Create the early stopping callback
    early_stop = EarlyStopping(rounds=61, metric_name="rmse")

    model = model_class(**params, objective='reg:squarederror', eval_metric='rmse', enable_categorical="True")

    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', ce.TargetEncoder(), categorical_features),
            ('cat_bin', OneHotEncoder(drop="first",sparse_output=False,handle_unknown="ignore"), cat_bin),
            ('num', StandardScaler(), numeric_features)
        ])

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)], verbose=False)

    kf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)
    rmse_scores = []


    fit_params = {'verbose': False}
    cv_scores = cross_val_score(model, X, y, cv=kf, scoring='neg_root_mean_squared_error', n_jobs=-1, params=fit_params)
    gc.collect()

    return np.mean(np.abs(cv_scores))

### A - XGBRegressor

In [ ]:
# Step 3: Tuning Hyperparameters with Optuna
def tune_hyperparameters(X, y, model_class, n_trials, use_gpu, n_splits = 1 ,n_repeats=1):
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())
    study.optimize(lambda trial: objective_xgb(trial, model_class, Xt, y, use_gpu=use_gpu, n_splits=n_splits, n_repeats=n_splits), n_trials=n_trials)
    return study  # Return the study object

# Step 5: Saving Best Results and Models
def save_results(study, model_class, model_name):
    best_params_file = f"{model_name}_best_params.joblib"
    joblib.dump(study.best_params, best_params_file)
    print(f"Best parameters for {model_name} saved to {best_params_file}")

    verbose_file = f"{model_name}_optuna_verbose.log"
    with open(verbose_file, "w") as f:
        f.write(str(study.trials))
    print(f"Optuna verbose for {model_name} saved to {verbose_file}")

#### Optuna Hyperameter Optimization:

In [ ]:
# usage with XGBRegressor
xgb_study = tune_hyperparameters(Xt, y, XGBRegressor, n_trials=51, use_gpu=False, n_splits = 3 ,n_repeats=3)
save_results(xgb_study, XGBRegressor, "XGBoost")
xgb_params = xgb_study.best_params

[I 2024-10-30 22:53:38,880] A new study created in memory with name: no-name-360d927c-f1c8-436e-8de0-092e351fd22b
[I 2024-10-30 23:00:18,215] Trial 0 finished with value: 0.47783102922969395 and parameters: {'max_depth': 18, 'min_child_weight': 1, 'colsample_bytree': 0.55, 'subsample': 0.7000000000000001, 'reg_lambda': 0.23164830432062045, 'reg_alpha': 0.00694765739707545, 'gamma': 0.08905614267354793}. Best is trial 0 with value: 0.47783102922969395.
[I 2024-10-30 23:03:13,796] Trial 1 finished with value: 0.47676679823133683 and parameters: {'max_depth': 14, 'min_child_weight': 5, 'colsample_bytree': 0.55, 'subsample': 0.875, 'reg_lambda': 0.003372245264215911, 'reg_alpha': 0.0002142866225420383, 'gamma': 2.9072499542044044e-05}. Best is trial 1 with value: 0.47676679823133683.
[I 2024-10-30 23:06:01,572] Trial 2 finished with value: 0.4819345573584239 and parameters: {'max_depth': 13, 'min_child_weight': 4, 'colsample_bytree': 0.75, 'subsample': 0.875, 'reg_lambda': 0.00099285991564

parameters: {'max_depth': 20, 'min_child_weight': 9, 'colsample_bytree': 0.675, 'subsample': 0.9500000000000001, 'reg_lambda': 0.0023578997094310383, 'reg_alpha': 0.0002511828751854158, 'gamma': 4.804072231366388e-05}. Best is trial 32 with value: 0.4603695107830895.

parameters: {'max_depth': 20, 'min_child_weight': 9, 'colsample_bytree': 0.7000000000000001, 'subsample': 0.925, 'reg_lambda': 0.012376109100807844, 'reg_alpha': 2.957509681090659e-05, 'gamma': 3.977885964317904e-05}. Best is trial 23 with value: 0.4604025748040941.

In [ ]:
print(xgb_params)